In [86]:
import pandas as pd
import numpy as np
import h5py
import matplotlib
import math
import random

#small_data = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/CS170_Small_Data__91.txt'
small_text = "/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/test/Sue/CS170_Small_Data__96.txt"

small_data_df = pd.read_csv(small_text,
                            sep ='  ',
                            engine = 'python',
                            header = None)

large_text = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/test/Sue/CS170_Large_Data__21.txt'


large_data_df = pd.read_csv(large_text, 
                         sep='  ',
                         engine = 'python',
                         header=None)


#small_arr = small_data
small_data = small_data_df.to_numpy()

large_data = large_data_df.to_numpy()


def stub(data_df,current_set,feature_to_add):
    random.seed(feature_to_add)
    return random.random()

def dist(a,b):
    diff = a[1:] - b[1:]
    return(np.dot(diff,diff))


def nn(data_df): #this is correct
    
    number_correctly_classified = 0
    
    for i in range(0,len(data_df)):
        #object_to_classify = data_df.iloc[i]
        object_to_classify = data_df[i,1:]
        #label_object_to_classify = data_df.iloc[i][0]
        label_object_to_classify = data_df[i,0]
        
        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        
        for k in range(0,len(data_df)):
            if k!=i:            #object_to_classify,data_df.iloc[k]
                #distance = dist(object_to_classify,data_df[k,1:])
                distance = dist(data_df[i],data_df[k])
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = k 
                    #data_df.iloc[nearest_neighbor_location][0]
                    nearest_neighbor_label = data_df[nearest_neighbor_location,0]
                    #print(nearest_neighbor_label)               
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified = number_correctly_classified +1 
        
    accuracy = number_correctly_classified / len(data_df)
    #print(accuracy)
    return accuracy

def leave_one_out_cross_validation_forward(data_df,current_set,feature_to_add):
    new_current = current_set[:]
    new_current.append(feature_to_add)
    
    drops = []
    x = 1
    for x in range(1,data_df.shape[1]): #iterate through the columns
        if x not in new_current: #drop that column if it is not inside of the current set
            drops.append(x)
            
        
    #data_df.loc[:,drops] = 0
    new_data_df = data_df.copy()
    new_data_df[:,drops]=0 #

    accuracy = nn(new_data_df)
    return accuracy

'''
current_set = [1]
feature_to_add = 2

print(leave_one_out_cross_validation(small_data,current_set,feature_to_add))
'''


        


'\ncurrent_set = [1]\nfeature_to_add = 2\n\nprint(leave_one_out_cross_validation(small_data,current_set,feature_to_add))\n'

In [ ]:
def forward_feature_selection(data):
    current_set_of_features = [] #these are the list with the best accuracies at each level
    best_accuracies=[]
    #relevant_feature_tuples = []
    for i in range(1,data.shape[1]):#level
        
        #print("On the ",i," level of the search tree")
        
        feature_to_add_at_this_level = []
        best_so_far_acc = 0
        
        for k in range(1,data.shape[1]):#features
            if k not in current_set_of_features:
                #print("Considering adding feature ",k)
                 
                accuracy = leave_one_out_cross_validation_forward(data,current_set_of_features,k)
                print("Feature(s) {",*current_set_of_features,k,"} accuracy is ",accuracy)
                if (accuracy > best_so_far_acc):
                    best_so_far_acc = accuracy
                    feature_to_add_at_this_level = k
        best_accuracies.append(best_so_far_acc)                     
        current_set_of_features.append(feature_to_add_at_this_level)
        #relevant_feature_tuples.append(current_set_of_features)
        print("On level ",i," I added feature",feature_to_add_at_this_level," to current set with accuracy",best_so_far_acc)
    print("These are our features",current_set_of_features)
    print("These are the accuracies",best_accuracies)
    #print("These are relevant feature combinations",relevant_feature_tuples)
        
forward_feature_selection(small_data)

In [ ]:
def copyArray(data):
    current = []
    for i in range(1,data.shape[1]):
        current.append(i)
    return current

#current = []
current = copyArray(large_data)
print(*current)

In [89]:
def leave_one_out_cross_validation_elimination(data_df,current_set,feature_to_remove):
    new_current = current_set[:]
    new_current.remove(feature_to_remove)
    
    drops = []
    x = 1
    for x in range(1,data_df.shape[1]): #iterate through the columns
        if x not in new_current: #drop that column if it is not inside of the current set
            drops.append(x)
            
        
    #data_df.loc[:,drops] = 0
    new_data_df = data_df.copy()
    new_data_df[:,drops]=0 #

    accuracy = nn(new_data_df)
    return accuracy

In [ ]:
def backward_elimination_feature_selection(data):
    current_set_of_features = copyArray(data)
    best_accuracies=[]
    #relevant_feature_tuples = []
    for i in range(1,data.shape[1]):#level
        
        print("On level ",i," of the search tree")
        
        feature_to_add_at_this_level = []
        best_so_far_acc = 0
        
        for k in range(1,data.shape[1]):#features
            if k in current_set_of_features:
                #print("Considering adding feature ",k)
                 
                accuracy = leave_one_out_cross_validation_elimination(data,current_set_of_features,k)
                print("Feature(s) {",*current_set_of_features,"} with removal of",k," Has accuracy:",accuracy)
                if (accuracy > best_so_far_acc):
                    best_so_far_acc = accuracy
                    feature_to_add_at_this_level = k
        best_accuracies.append(best_so_far_acc)                     
        current_set_of_features.remove(feature_to_add_at_this_level)
        #relevant_feature_tuples.append(current_set_of_features)
        print("On level ",i," I removed feature",feature_to_add_at_this_level," from the current set with accuracy",best_so_far_acc)
    print("These are our features",current_set_of_features)
    print("These are the accuracies",best_accuracies)

backward_elimination_feature_selection(small_data)